# SUDOKU SOLVER

In [1]:
# Including path to previous directory in built-in variable sys.path

import sys

sys.path.append('../')

Importing the relevant libraries:

In [3]:
import numpy as np
from sudoku.utils import N, STEP, objective_grid

Definition of auxiliary functions:

In [4]:
def square_loc(pos):
    pos_square = pos // STEP * STEP
    
    return pos_square, pos_square + STEP

## Defining CSP

Class that defines the CSP and allows the application of the solver:

In [5]:
class SudokuCSP:
    def __init__(self, grid):
        self.domains = self.node_consistency(grid)
                
        self.neighbors = self.generate_neighbors()  # avoid checking each time it is necessary
        
        self.constraints = self.generate_constraints()
                
    @staticmethod
    def node_consistency(grid):
        domains = {(y, x) : [grid[y, x]] if grid[y, x] else list(range(1, N+1)) 
                   for y in range(N)
                   for x in range(N)}
        
        return domains
    
    def generate_neighbors(self):
        neighbors = {(y, x) : set([(y, i) for i in range(N) if i != x] + 
                                  [(j, x) for j in range(N) if j != y] + 
                                  [(j, i) for j in range(*square_loc(y)) for i in range(*square_loc(x))
                                   if i != x and j != y])
                     for y, x in self.vars}
        
        return neighbors
    
    def generate_constraints(self):
        return [(var, neighbor) for var in self.vars for neighbor in self.neighbors[var]]
    
    def ac_3(self):
        queue = self.constraints.copy()
        
        while queue:
            xi, xj = queue.pop(0)
            
            if self.revise(xi, xj):
                if not self.domains[xi]:
                    return False
                
                for xk in self.neighbors[xi] - {xj}:
                    queue.append((xk, xi))
        
        return True
    
    def revise(self, xi, xj):
        revised = False
        
        for x in self.domains[xi]:
            if not any([x != y for y in self.domains[xj]]):
                self.domains[xi].remove(x)
                
                revised = True
        
        return revised
        
    @property
    def vars(self):
        return list(self.domains.keys())
    
    @property
    def won(self):
        if all(list(map(lambda x: len(x) == 1, csp.domains.values()))):
            grid = np.array(list(map(lambda domain: domain[0], csp.domains.values()))).reshape((N, N))
            
            return objective_grid(grid)
        
        return False

Testing class instantiation and resolution through AC-3:

In [6]:
# Sudoku that can only be solved with AC-3
grid_ac3 = np.array([[1, 5, 7, 4, 0, 8, 9, 3, 0],
                     [8, 0, 3, 0, 0, 5, 0, 0, 4],
                     [6, 0, 4, 0, 1, 9, 8, 0, 7],
                     [0, 6, 0, 7, 0, 0, 0, 0, 0],
                     [0, 0, 0, 6, 0, 4, 2, 0, 3],
                     [0, 3, 9, 5, 0, 1, 0, 0, 0],
                     [5, 0, 1, 8, 0, 0, 3, 0, 9],
                     [3, 8, 0, 0, 5, 6, 0, 4, 0],
                     [0, 0, 6, 0, 3, 7, 0, 2, 0]], dtype='int8')

In [7]:
csp = SudokuCSP(grid_ac3)

csp.domains

{(0, 0): [1],
 (0, 1): [5],
 (0, 2): [7],
 (0, 3): [4],
 (0, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 5): [8],
 (0, 6): [9],
 (0, 7): [3],
 (0, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 0): [8],
 (1, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 2): [3],
 (1, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 5): [5],
 (1, 6): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 8): [4],
 (2, 0): [6],
 (2, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 2): [4],
 (2, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 4): [1],
 (2, 5): [9],
 (2, 6): [8],
 (2, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 8): [7],
 (3, 0): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 1): [6],
 (3, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 3): [7],
 (3, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 6): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (4, 0): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (4, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (4, 2): [1,

In [8]:
# Performing the propagation of inferences:

csp.ac_3()

True

In [9]:
csp.won

True

Testing class instantiation and resolution via backtracking search:

In [10]:
# Sudoku that cannot be solved with AC-3 alone
grid_backtracking = np.array([[0, 0, 5, 3, 0, 0, 0, 0, 0],
                              [8, 0, 0, 0, 0, 0, 0, 2, 0],
                              [0, 7, 0, 0, 1, 0, 5, 0, 0],
                              [4, 0, 0, 0, 0, 5, 3, 0, 0],
                              [0, 1, 0, 0, 7, 0, 0, 0, 6],
                              [0, 0, 3, 2, 0, 0, 0, 8, 0],
                              [0, 6, 0, 5, 0, 0, 0, 0, 9],
                              [0, 0, 4, 0, 0, 0, 0, 3, 0],
                              [0, 0, 0, 0, 0, 9, 7, 0, 0]], dtype='int8')

In [11]:
csp = SudokuCSP(grid_backtracking)

csp.domains

{(0, 0): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 2): [5],
 (0, 3): [3],
 (0, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 6): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 0): [8],
 (1, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 6): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 7): [2],
 (1, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 0): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 1): [7],
 (2, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 4): [1],
 (2, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 6): [5],
 (2, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 0): [4],
 (3, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 4): [1, 2

In [12]:
csp.ac_3()

True

In [13]:
csp.won

False